In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
df = pd.read_excel("5295388-E9A(06.16-05.17).XLSX")

In [ ]:
df.columns

In [ ]:
df1 = df[['Doc. Date', 'Order Quantity']]

In [ ]:
df2 = pd.read_excel("5295388-E9A(06.17-05.18).XLSX")

In [ ]:
df2_1 = df2[['Doc. Date', 'Order Quantity']]

In [ ]:
df3 = pd.read_excel("5295388-E9A(06.18-05.19).XLSX")

In [ ]:
df3_1 = df3[['Doc. Date', 'Order Quantity']]

In [ ]:
data = df1.append([df2_1, df3_1])

In [ ]:
data.head(20)

In [ ]:
df_final = data.groupby('Doc. Date')['Order Quantity'].sum().reset_index()

In [ ]:
df_final.shape

In [ ]:
df_final.head(40)

In [ ]:
df_final.columns

In [ ]:
df_final['Doc. Date'].min(), df_final['Doc. Date'].max()

In [ ]:
data.isnull().sum() # check null data

In [ ]:
df_data = df_final.set_index('Doc. Date')

In [ ]:
df_data.index

In [ ]:
median_ordered_qty = df_data['Order Quantity'].median()
median_ordered_qty

In [ ]:

y = df_data['Order Quantity'].resample('W').sum()

In [ ]:
type(y)

In [ ]:
y.tail(10)

In [ ]:
y.isnull().value_counts()

In [ ]:
y = y.replace('NaN',median_ordered_qty)

In [ ]:
y.isnull().value_counts()

In [ ]:
y.head()

In [ ]:
y.plot(figsize=(15, 6))
plt.show

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8

In [ ]:
decomposition = sm.tsa.seasonal_decompose(y, model='additive')
fig = decomposition.plot()
plt.show()

# Time series forecasting with ARIMA

In [ ]:
p = d = q = range(0, 2)

In [ ]:
pdq = list(itertools.product(p, d, q)) #cartesian product, equivalent to a nested for-loop

In [ ]:
type(pdq)

In [ ]:
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]

In [ ]:
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}52 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 0, 52),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

In [ ]:
# run model diagnostics to investigate any unusual behavior

In [ ]:
results.plot_diagnostics(figsize=(32, 16))
#plt.show()

model diagnostics suggests that the model residuals are near normally distributed

# Validating forecasts

In [ ]:
y['2019':]

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2019-01-06'), dynamic=False)
pred_ci = pred.conf_int()

ax = y['2017':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=0.7, figsize=(14, 7))

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Week')
ax.set_ylabel('Quantity')
plt.legend()

plt.show()



In [ ]:
y_forecasted = pred.predicted_mean
y_truth = y['2019-01-06':]

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

In [ ]:
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))